# COVID-19 Detection from Chest X-rays using ResNet-18

# 1. Problem Statment
Early and accurate diagnosis of COVID-19 is crucial to contain its spread and initiate timely treatment. The standard diagnostic methods such as nasal and throat swabs (RT-PCR tests) often suffer from low sensitivity and delayed results, leading to false negatives and uncontrolled transmission. This project aim to develop an intelligent framework using **deep learning**, specifically the **ResNet-18** convolutional neural network, to:

Automatically classify **chest X-ray images** into three categories:
  - **COVID-19**
  - **Pneumonia (non-COVID)**
  - **Normal (Healthy)**

# 2. Imports 

In [2]:
%matplotlib inline

import os
import shutil
import random
import torch
import torchvision
import numpy as np

from PIL import Image
from matplotlib import pyplot as plt

torch.manual_seed(0)

print('Using PyTorch version', torch.__version__)

Using PyTorch version 2.7.1+cpu


# 3. preprocess the train and test data

In [ ]:

class_names = ['normal', 'viral', 'covid']
root_dir = 'COVID-19 Radiography Database'
source_dirs = ['NORMAL', 'Viral Pneumonia', 'COVID-19']

if os.path.isdir(os.path.join(root_dir, source_dirs[1])):
    os.mkdir(os.path.join(root_dir, 'test'))

    for i, d in enumerate(source_dirs):
        os.rename(os.path.join(root_dir, d), os.path.join(root_dir, class_names[i]))

    for c in class_names:
        os.mkdir(os.path.join(root_dir, 'test', c))

    for c in class_names:
        images = [x for x in os.listdir(os.path.join(root_dir, c)) if x.lower().endswith('png')]
        selected_images = random.sample(images, 30)
        for image in selected_images:
            source_path = os.path.join(root_dir, c, image)
            target_path = os.path.join(root_dir, 'test', c, image)
            shutil.move(source_path, target_path)

'NORMAL'